# 选择pytorch的原因
PyTorch的基本单位是张量(Tensor)，类似于python中的“numpy”数组。使用PyTorch有很多好处，但其最重要的两个是：

* 动态网络–在运行期间可以改变网络架构
* 跨GPU的分布式训练

## 1.处理超出词汇表的单词
在固定的词汇表上训练文本分类模型。但是在训练过程中，我们可能会遇到一些词汇表中不存在的单词。这些单词被称为词汇表外单词。超出词汇表单词的出现可能是一个关键问题，因为这会导致信息丢失。

为了处理超出词汇表的单词，PyTorch支持一项很酷的功能，即用未知令牌(token)来替换训练数据中的出现次数少的单词。反过来，这将有助于我们解决单词超出词汇表的问题。

## 2.处理可变长度的序列
PyTorch带有一个有用的功能，即“填充序列(Packed Padding sequence)”，这个功能可以实现动态循环神经网络。

填充是指在句子的开头或结尾添加一个填充令牌的额外令牌的过程。随着每个句子中单词数量的变化，我们通过添加padding标记将变长度的输入句子转换为具有相同长度的句子。

由于大多数框架都支持静态网络，即在整个模型训练过程中，神经网络体系结构保持不变，因此需要进行填充。尽管填充解决了可变长度序列的问题，但是这个想法还有另一个问题–神经网络体系结构像处理其他信息/数据一样处理这些填充令牌。让我通过一个简单的图来解释这一点。

如您在下图中所见，在生成输出时，还需要使用最后一个元素（即填充令牌）。这可以通过PyTorch中的填充序列来解决

## 3.包装器和预训练模型
目前正在为PyTorch框架推出最先进的架构。Hugging Face发布了transformer，为自然语言理解提供了超过32种最先进的体系结构！

不仅如此，PyTorch还为一些任务提供了预训练的模型，例如文本转语音，对象检测等，用几行代码就可以实现。

# 问题陈述
Quora希望在其平台上跟踪用户不真诚的问题，以使用户在共享知识时感到安全。在这种情况下，一个不真诚的问题被定义为一个旨在陈述而不是在寻求有用答案的问题。为了进一步说明这一点，这里有一些特征可以表明一个特定的问题是不真诚的：

- 具有中立的语气
- 贬低或煽动
- 不扎根于现实
- 使用性内容（*****）来让人感到震惊，而不是寻求真正的答案

训练数据包括所提出的问题以及一个标志，该标志表示该问题是否被识别为不真诚（目标= 1）。真实标签包含一些噪音，即不能保证它们是完美的。我们的任务是在给定的问题下，判断该问题是否具有“诚意”。

In [1]:
import torch
from torchtext import data

In [2]:
# 为了使结果可以重复，我指定了随机种子的数值。由于深度学习模型的随机性，
# 在执行时可能会产生不同的结果，因此指定随机种子的数值很重要。

#产生相同的结果
SEED=2019

#Torch
torch.manual_seed(SEED)

#Cuda算法
torch.backends.cudnn.deterministic=True

## 数据预处理

有两种不同类型的字段对象–Field和LabelField。让我们快速了解两者之间的区别-

* Field：数据模块中的Field对象用于为数据集中的每一列指定预处理步骤。
* LabelField：LabelField对象是Field对象的特例，仅用于分类任务。它的唯一用途是将unk_token按顺序设置为默认值（None）。

在使用Field之前，让我们看一下Field的不同参数以及它们的用途。Field的参数：

* Tokenize：指定句子的令牌化(tokenize)方式，即转换将句子转换为单词的方式。我正在使用spacy令牌生成器，因为它使用了新颖的令牌生成算法
* Lower: 将文本转换为小写
* batch_first: 输入和输出的第一维始终是批处理大小(batch size)

In [3]:
# TEXT=data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
# LABEL=data.LabelField(dtype=torch.float,batch_first=True)
# from spacy.lang.en import English
#切记加上tokenizer_language='en_core_web_sm',默认为en
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True,tokenizer_language='en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

### 大家再上面如果不加上tokenizer_language='en_core_web_sm'的话，会报错没有en这个库，应该是由于spacy的更新，所以使用的预训练模型出现了问题

接下来，我们将创建一个元组列表，其中每个元组中的第一个值包含一个列名，第二个值是上面定义的字段对象。此外，我们将按照csv的列的排列顺序，来排列每个元组，并指定为（None，None）以忽略csv文件中的列。

In [4]:
# 让我们只读取需要的列-问题和标签
fields=[(None,None),('text',TEXT),('label',LABEL)]

In [5]:
import chardet
# 在下面的代码块中，我通过定义字段对象加载自定义数据集。
training_data=data.TabularDataset(path='quora.csv',format='csv',fields=fields,skip_header=True)

#打印预处理文本
print(vars(training_data.examples[0]))

{'text': ['Why', 'are', 'most', 'indian', 'parents', 'against', 'even', 'liking', 'someone', '?'], 'label': '1'}


In [6]:
import random
train_data,valid_data=training_data.split(split_ratio=0.7,random_state=random.seed(SEED))

## 准备输入和输出序列
下一步是构建词汇表并将文本转换为整数序列。词汇表包含文本数据中的单词。每个单词都分配有一个索引。以下是参数
参数：

- 1. min_freq：忽略频率小于指定频率的单词，并将其映射到未知令牌。
- 2. 两个特殊的令牌，unknown和padding，它们也将添加到词汇表中
* Unknown令牌用于处理超出词汇表的单词
* Padding令牌用于制作相同长度的输入序列
让我们建立词汇表并使用预训练的词嵌入来初始化单词。如果您希望随机初始化嵌入，请忽略vectors参数。

In [7]:
#初始化glove词嵌入
TEXT.build_vocab(train_data,min_freq=3,vectors="glove.6B.100d")
LABEL.build_vocab(train_data)

#文本中的唯一标记
print("Size of TEXT vocabulary:" ,len(TEXT.vocab))

#标签中唯一令牌的集合
print("Size of LABEL vocabulary:" ,len(LABEL.vocab))

#常用单词
print(TEXT.vocab.freqs.most_common(10))

#单词词典
print(TEXT.vocab.stoi)

Size of TEXT vocabulary: 17126
Size of LABEL vocabulary: 2
[('?', 76268), ('the', 39173), ('to', 26235), ('a', 22241), (',', 20534), ('of', 19236), ('in', 18874), ('and', 18403), ('Why', 17354), ('is', 17305)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x0000019DF9461CC8>>, {'<unk>': 0, '<pad>': 1, '?': 2, 'the': 3, 'to': 4, 'a': 5, ',': 6, 'of': 7, 'in': 8, 'and': 9, 'Why': 10, 'is': 11, 'do': 12, 'What': 13, 'I': 14, 'are': 15, 'How': 16, 'for': 17, 'that': 18, 'it': 19, 'you': 20, '.': 21, 'Is': 22, 'people': 23, 'with': 24, 'have': 25, 'they': 26, 'be': 27, 'on': 28, 'or': 29, 'can': 30, '"': 31, 'my': 32, 'so': 33, "n't": 34, '-': 35, 'their': 36, 'as': 37, "'s": 38, 'not': 39, 'does': 40, 'from': 41, 'like': 42, 'Do': 43, 'when': 44, 'if': 45, 'an': 46, 'about': 47, ')': 48, 'by': 49, 'get': 50, '(': 51, 'there': 52, 'your': 53, 'Trump': 54, 'women': 55, 'would': 56, 'Are': 57, 'all': 58, 'who': 59, 'If': 60, 'did': 61, 'will': 62, 'at'

In [8]:
LABEL.vocab.stoi

defaultdict(None, {'0': 0, '1': 1})

In [9]:
# 现在，我们将准备用于训练模型的批数据(batch)。BucketIterator以需要最少的填充量
# 的方式来形成批数据(batch)。
#检查cuda是否可用
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#设置批数据大小
BATCH_SIZE=64

#加载迭代器
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

# 模型架构

**现在是时候定义解决二分类问题的模型结构了。torch的nn模块是所有模型的基本模型。这意味着每个模型都必须是nn模块的子类。**

我在这里定义了2个函数：init和forward。让我解释一下这两个函数的用法：

* 1.Init：每当创建类的实例时，都会自动调用init函数。因此，它被称为构造函数。传递给类的参数由构造函数初始化。我们在该方法中定义模型将使用的所有层。

* 2.Forward：定义输入数据的正向传递。

最后，让我们详细了解用于构建模型架构的不同层及其参数

**嵌入层(Embedding layer)**：嵌入对于任何与NLP相关的任务都非常重要，因为它以数字格式表示单词。嵌入层创建一个查找表，其中每一行代表一个单词的嵌入。嵌入层将整数序列转换为密集的矢量表示形式。这是嵌入层的两个最重要的参数

- 1.num_embeddings：词汇表单词的数量
- 2.embedding_dim：一个单词表示的维数

**LSTM**：LSTM是RNN的变体，能够捕获长期依赖关系。您应该熟悉的LSTM一些重要的参数。以下是该层的参数：

- 1.input_size：输入的维数
- 2.hidden_size：隐藏节点数
- 3.num_layers：要堆叠的层数
- 4.batch_first：如果为True，则输入和输出张量按（batch，seq，feature）提供
- 5.dropout：如果不为零，则在除最后一层以外的每个LSTM层的输出上都加入一个Dropout层，其丢弃概率等于dropout的值。默认值：0
- 6.bidirection：如果为True，则使用双向的LSTM

**线性层(Linear Layer)**：线性层是指全连接层(dense层)。此处描述了两个重要参数：

- 1.in_features：输入特征的数量
- 2.out_features：输出特征的数量

**Pack Padding**：像之前所讨论的那样，pack padding用于定义动态循环神经网络。如果没有pack padding，填充输入的令牌也将由rnn处理，并返回填充令牌的隐藏状态。这是一个很棒的包装器，不显示输入的填充。它只是忽略这些值并返回未填充令牌的隐藏状态。

In [10]:
# 定义架构的所有层
import torch.nn as nn
class classifier(nn.Module):
    #定义模型中使用的所有层
    def __init__(self,vocab_size,embedding_dim,hidden_dim,output_dim,n_layers,
                bidirectional,dropout):
        #构造函数
        super().__init__()
        
        #embedding层
        self.embedding=nn.Embedding(vocab_size,embedding_dim)
        
        #lstm层
        self.lstm=nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=n_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True
        )
        #dense层
        self.fc=nn.Linear(hidden_dim*2,output_dim)
        
        #激活函数
        self.act=nn.Sigmoid()
    def forward(self,text,text_lengths):
        
        #text=[batch size.sent_length]
        embedded=self.embedding(text)
        #embedded=[batch size,sent_len,emb dim]
        
        #填充句子
        packed_embedded=nn.utils.rnn.pack_padded_sequence(embedded,text_lengths,batch_first=True)
        packed_output,(hidden,cell)=self.lstm(packed_embedded)
        #hidden=[batch size,num layers*num directions.hid dim]
        #cell=[batch size,num layers*num directions,hid dim]
        
        #合并（concat）前向传播和后向传播的最终隐藏状态
        hidden=torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim=1)
        
        #hidden=[batch size,hid dim*num directions]
        dense_outputs=self.fc(hidden)
        
        #最终激活函数
        outputs=self.act(dense_outputs)
        
        return outputs

In [11]:
# 下一步是定义超参数并实例化模型。
#定义超参数
size_of_vocab=len(TEXT.vocab)
embedding_dim=100
num_hidden_nodes=32
num_output_nodes=1
num_layers=2
bidirection=True
dropout=0.2

#定义完超参数之后，使用这些定义好的超参数实例化本问题专属的模型
model=classifier(size_of_vocab,embedding_dim,num_hidden_nodes,num_output_nodes,
                num_layers,bidirectional=True,dropout=dropout)

In [12]:
# 看一下模型摘要(summary)，并使用预训练的词嵌入来初始化嵌入层
#模型框架
print(model)

#可训练参数的数量
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

#初始化预训练的词嵌入
pretrained_embeddings=TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(17126, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 1,772,057 trainable parameters
torch.Size([17126, 100])


In [13]:
# 为模型定义了优化器，损失(loss)和度量指标
import torch.optim as optim

#定义优化器和损失
#告知优化器需要优化的参数
optimizer=optim.Adam(model.parameters())
criterion=nn.BCELoss()

#定义度量指标
def binary_accuracy(preds,y):
    #round预测到最接近的整数
    rounded_preds=torch.round(preds)
    
    correct=(rounded_preds==y).float()
    acc=correct.sum()/len(correct)
    return acc
#转化为cuda（如果可用）
model=model.to(device)
criterion=criterion.to(device)

## 构建模型有两个阶段：

- 1.训练阶段：model.train()将模型设置为训练阶段并激活dropout层。
- 2.测试阶段：model.eval()将模型设置为评估阶段并停用dropout层。

In [14]:
# 用于定义训练模型的函数
from tqdm import tqdm
def train(model,iterator,optimizer,criterion):
    #每个epoch进行初始化
    epoch_loss=0
    epoch_acc=0
    
    #将模型设置为训练阶段
    model.train()
    
    for batch in tqdm(iterator):
        #重设梯度
        optimizer.zero_grad()
        
        #获取文本和单词数量
        text,text_lengths=batch.text
        
        #转换为一维向量
        predictions=model(text,text_lengths).squeeze()
        
        #计算loss
        loss=criterion(predictions,batch.label)
        
        #计算二分类准确度
        acc=binary_accuracy(predictions,batch.label)
        
        #后向传播损失并计算梯度
        loss.backward()
        
        #更新权重
        optimizer.step()
        
        #损失和准确度
        epoch_loss+=loss.item()
        epoch_acc+=acc.item()
    return epoch_loss/len(iterator),epoch_acc/len(iterator)

In [15]:
#用于评估模型

def evaluate(model,iterator,criterion):
    #每个epoch进行初始化
    epoch_loss=0
    epoch_acc=0
    
    #停用dropout层
    model.eval()
    
    #停用自动求导
    with torch.no_grad():
        
        for batch in tqdm(iterator):
            
            #获取文本和单词数量
            text,text_lengths=batch.text
            
            #转换为一维张量
            predictions=model(text,text_lengths).squeeze()
            
            #计算损失和准确度
            loss=criterion(predictions,batch.label)
            acc=binary_accuracy(predictions,batch.label)
            
            #跟踪损失和准确度
            epoch_loss+=loss.item()
            epoch_acc+=acc.item()
        return epoch_loss/len(iterator),epoch_acc/len(iterator)

In [16]:
# 最后，我们在一定时间内训练模型，并在每个时间段保存最佳模型。
from tqdm import tqdm
N_EPOCHS=5
best_valid_loss=float('inf')

for epoch in range(N_EPOCHS):
    #训练模型
    train_loss,train_acc=train(model,train_iterator,optimizer,criterion)
    
    #评估模型
    valid_loss,valid_acc=evaluate(model,valid_iterator,criterion)
    
    #保存模型
    if valid_loss<best_valid_loss:
        best_valid_loss=valid_loss
        torch.save(model.state_dict(),'saved_weights.pt')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |   Val.Acc: {valid_acc*100:.2f}%')

  0%|                                                                                         | 0/1092 [00:00<?, ?it/s]

	Train Loss: 0.333 | Train Acc: 85.98%
	 Val. Loss: 0.278 |   Val.Acc: 89.16%


  0%|                                                                                         | 0/1092 [00:00<?, ?it/s]

	Train Loss: 0.243 | Train Acc: 90.73%
	 Val. Loss: 0.279 |   Val.Acc: 88.99%


  0%|                                                                                         | 0/1092 [00:00<?, ?it/s]

	Train Loss: 0.206 | Train Acc: 92.34%
	 Val. Loss: 0.288 |   Val.Acc: 89.13%


  0%|                                                                                         | 0/1092 [00:00<?, ?it/s]

	Train Loss: 0.172 | Train Acc: 93.78%
	 Val. Loss: 0.310 |   Val.Acc: 88.62%


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:09<00:00, 51.70it/s]

	Train Loss: 0.140 | Train Acc: 94.99%
	 Val. Loss: 0.337 |   Val.Acc: 88.16%


In [20]:
# 让我们加载最佳模型并定义推断函数，该函数接受用户定义的输入并进行预测
# 加载权重
path='saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();

#推断 
import spacy
nlp = spacy.load('en_core_web_sm')

def predict(model, sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  #令牌化(tokenize)句子 
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]          #转换为整数序列
    length = [len(indexed)]                                    #计算单词个数
    tensor = torch.LongTensor(indexed).to(device)              #转换为张量
    tensor = tensor.unsqueeze(1).T                             #reshape成[batch, 单词个数]
    length_tensor = torch.LongTensor(length)                   #转换为张量
    prediction = model(tensor, length_tensor)                  #预测
    return prediction.item()


In [23]:
# 惊人！让我们使用此模型对几个问题进行预测
#进行预测
predict(model, "Are there any sports that you don't like?")

#不真诚的问题
predict(model, "Why Indian girls go crazy about marrying Shri. Rahul Gandhi ji?")

0.9737265110015869

In [ ]:
import chardet
# chardet.detect(open('quora.csv','rb'))
with open('quora.csv',"rb") as f:
    bytes=f.read()
    print(chardet.detect(bytes))

In [ ]:
import chardet
# chardet.detect(open('quora.csv','rb'))
with open('quora1.csv',"rb") as f:
    bytes=f.read()
    print(chardet.detect(bytes))